In [ ]:
import os

import pinecone
from dotenv import load_dotenv
from langchain.agents import AgentType
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import PythonLoader
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import Language, RecursiveCharacterTextSplitter
from langchain.vectorstores import Pinecone
from langchain_experimental.agents.agent_toolkits import create_python_agent
from langchain_experimental.tools import PythonREPLTool

load_dotenv()


# initialize Vector DB
pinecone.init(
    api_key=os.environ.get("PINECONE_API_KEY"),
    environment=os.environ.get("PINECONE_ENV"),
)

# set Index Params
# note: dimension of the index is found from the embeddings model
# site: https://platform.openai.com/docs/guides/embeddings/what-are-embeddings
# model: text-embedding-ada-002

index_name = "ai-developer"
index_dim = 1536
index_metric = "euclidean"

if index_name in pinecone.list_indexes():
    index = pinecone.Index(index_name)

    # if the vectors are present in index - clean up & create a new index
    # no better way due to free tier
    if index.describe_index_stats()["total_vector_count"] > 0:
        pinecone.delete_index(name=index_name)
        pinecone.create_index(name=index_name, dimension=index_dim, metric=index_metric)
    else:
        pass
else:
    # if index not present - create one
    pinecone.create_index(name=index_name, dimension=index_dim, metric=index_metric)